In [1]:
import numpy as np
import os
import sys
import time
import argparse
import json
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

import torch
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable

sys.path.insert(0,'../modules')
from sample_generator import *
from data_prov import *
from model import *
from bbreg import *
from options import *
from gen_config import *

In [2]:
np.random.seed(123)
torch.manual_seed(456)
torch.cuda.manual_seed(789)

In [3]:
def find_k(co1,co2):
    x1 = co1[0]
    y1 = co1[1]
    x2 = co2[0]
    y2 = co2[1]
    if (x2-x1)!=0:
        k = np.absolute((y2 - y1) / (x2 - x1))
    else:
        k = -1
    return k

In [4]:
def find_line(co1,co2):
    x1 = co1[0]
    y1 = co1[1]
    x2 = co2[0]
    y2 = co2[1]
    k = (y2-y1)/(x2-x1)
    b = y1 - k * x1
    return k,b
    

In [5]:
def find_distance(k,b,co_cluster):
    N = len(co_cluster)
    sum_dis = 0
    for i in range(N):
        x = co_cluster[i][0]
        y = co_cluster[i][1]
        d = np.absolute(k*x-y+b) / np.sqrt(k*k+1)
        #print(d)
        sum_dis = sum_dis + d
    sum_dis = sum_dis / N
    return sum_dis    
        

In [6]:
def find_offensive(object_index,cluster,threshold):
    m = object_index
    offensive = False
    N = len(cluster[m])
    if N < 11 or N >80:
        offensive = False
    else:
        k_sum=0
        k_interval = find_k(cluster[m][0],cluster[m][1])
        for i in range (1, N-2):
            temp = find_k(cluster[m][i],cluster[m][i+1])
            if temp > 0:
                k_current = temp
                k_difference = np.absolute(k_current - k_interval)
                k_interval = k_current
                k_sum = k_sum + k_difference
        k_sum = k_sum / (N - 2)
        if k_sum < threshold:
            offensive = False
        else:
            offensive = True
        print(k_sum)
    
    return offensive       

In [7]:
def find_offensive_V2(object_index,cluster,threshold):
    m = object_index
    offensive = False
    N = len(cluster[m])
    if N < 10:
        offensive = False
    else:
        p1 = cluster[m][0]
        p2 = cluster[m][N-1]
        k,b = find_line(p1,p2)
        sum_dis = find_distance(k,b,cluster[m])
        print("sum_dis = "+str(sum_dis))
        if sum_dis > threshold:
            offensive = True
        else:
            offensive = False
    return offensive       

In [8]:
def forward_samples(model, image, samples, out_layer='conv3'):
    model.eval()
    extractor = RegionExtractor(image, samples, opts['img_size'], opts['padding'], opts['batch_test'])
    for i, regions in enumerate(extractor):
        regions = Variable(regions)
        if opts['use_gpu']:
            regions = regions.cuda()
        feat = model(regions, out_layer=out_layer)
        if i==0:
            feats = feat.data.clone()
        else:
            feats = torch.cat((feats,feat.data.clone()),0)
    return feats

In [9]:
def set_optimizer(model, lr_base, lr_mult=opts['lr_mult'], momentum=opts['momentum'], w_decay=opts['w_decay']):
    params = model.get_learnable_params()
    param_list = []
    for k, p in params.iteritems():
        lr = lr_base
        for l, m in lr_mult.iteritems():
            if k.startswith(l):
                lr = lr_base * m
        param_list.append({'params': [p], 'lr':lr})
    optimizer = optim.SGD(param_list, lr = lr, momentum=momentum, weight_decay=w_decay)
    return optimizer

In [10]:
def train(model, criterion, optimizer, pos_feats, neg_feats, maxiter, in_layer='fc4'):
    model.train()
    
    batch_pos = opts['batch_pos']
    batch_neg = opts['batch_neg']
    batch_test = opts['batch_test']
    batch_neg_cand = max(opts['batch_neg_cand'], batch_neg)

    pos_idx = np.random.permutation(pos_feats.size(0))
    neg_idx = np.random.permutation(neg_feats.size(0))
    while(len(pos_idx) < batch_pos*maxiter):
        pos_idx = np.concatenate([pos_idx, np.random.permutation(pos_feats.size(0))])
    while(len(neg_idx) < batch_neg_cand*maxiter):
        neg_idx = np.concatenate([neg_idx, np.random.permutation(neg_feats.size(0))])
    pos_pointer = 0
    neg_pointer = 0

    for iter in range(maxiter):

        # select pos idx
        pos_next = pos_pointer+batch_pos
        pos_cur_idx = pos_idx[pos_pointer:pos_next]
        pos_cur_idx = pos_feats.new(pos_cur_idx).long()
        pos_pointer = pos_next

        # select neg idx
        neg_next = neg_pointer+batch_neg_cand
        neg_cur_idx = neg_idx[neg_pointer:neg_next]
        neg_cur_idx = neg_feats.new(neg_cur_idx).long()
        neg_pointer = neg_next

        # create batch
        batch_pos_feats = Variable(pos_feats.index_select(0, pos_cur_idx))
        batch_neg_feats = Variable(neg_feats.index_select(0, neg_cur_idx))

        # hard negative mining
        if batch_neg_cand > batch_neg:
            model.eval()
            for start in range(0,batch_neg_cand,batch_test):
                end = min(start+batch_test,batch_neg_cand)
                score = model(batch_neg_feats[start:end], in_layer=in_layer)
                if start==0:
                    neg_cand_score = score.data[:,1].clone()
                else:
                    neg_cand_score = torch.cat((neg_cand_score, score.data[:,1].clone()),0)

            _, top_idx = neg_cand_score.topk(batch_neg)
            batch_neg_feats = batch_neg_feats.index_select(0, Variable(top_idx))
            model.train()
        
        # forward
        pos_score = model(batch_pos_feats, in_layer=in_layer)
        neg_score = model(batch_neg_feats, in_layer=in_layer)
        
        # optimize
        loss = criterion(pos_score, neg_score)
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), opts['grad_clip'])
        optimizer.step()

        #print "Iter %d, Loss %.4f" % (iter, loss.data[0])

In [11]:
def save_fig(image, trbb, gtbb, ind, j, jF, type = 1):
    Font1 = ImageFont.truetype('/usr/share/fonts/opentype/noto/NotoSansCJK-Black.ttc',60)
    draw = ImageDraw.Draw(image)
    for m in range(trbb.shape[0]):
        if jF[m] == 0:
            continue
        if j[m] == 0:
            draw.line([trbb[m, 0], trbb[m, 1], trbb[m, 0], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1], trbb[m, 0], trbb[m, 1]], fill = (0,0,255), width = 6)
            draw.text([trbb[m, 0] + 2, trbb[m, 1] + 2], str(m + 1), font = Font1, fill = (0,0,255))
            if type is 1:
                draw.line([gtbb[m, 0], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1]], fill = (255,0,0), width = 6)
        else:
            draw.line([trbb[m, 0], trbb[m, 1], trbb[m, 0], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1], trbb[m, 0], trbb[m, 1]], fill = (0,255,0), width = 6)
            draw.text([trbb[m, 0] + 2, trbb[m, 1] + 2], str(m + 1), font = Font1, fill = (0,0,255))
            if type is 1:
                draw.line([gtbb[m, 0], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1]], fill = (255,0,0), width = 6) 
    del draw
    image.save(os.path.join(savefig_dir,'%04d.jpg'%(ind)), "JPEG")
    del image

In [12]:
def save_fig_online(image, trbb, gtbb, ind, j, jF, type = 1,cluster=[],thres=40):
    Font1 = ImageFont.truetype('/usr/share/fonts/opentype/noto/NotoSansCJK-Black.ttc',60)
    draw = ImageDraw.Draw(image)
    for m in range(trbb.shape[0]):
        if jF[m] == 0:
            continue
        if j[m] == 0:
            if find_offensive_V2(m,cluster,thres) == False:
                draw.line([trbb[m, 0], trbb[m, 1], trbb[m, 0], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1], trbb[m, 0], trbb[m, 1]], fill = (0,0,255), width = 6)
                draw.text([trbb[m, 0] + 2, trbb[m, 1] + 2], str(m + 1), font = Font1, fill = (0,0,255))
                if type is 1:
                    draw.line([gtbb[m, 0], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1]], fill = (0,0,255), width = 6)
            else:
                draw.line([trbb[m, 0], trbb[m, 1], trbb[m, 0], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1], trbb[m, 0], trbb[m, 1]], fill = (0,0,255), width = 6)
                draw.text([trbb[m, 0] + 2, trbb[m, 1] + 2], str(m + 1), font = Font1, fill = (0,0,255))
                if type is 1:
                    draw.line([gtbb[m, 0], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1]], fill = (0,0,255), width = 6)             
        else:
            if find_offensive_V2(m,cluster,thres) == False:
                draw.line([trbb[m, 0], trbb[m, 1], trbb[m, 0], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1], trbb[m, 0], trbb[m, 1]], fill = (0,255,0), width = 6)
                draw.text([trbb[m, 0] + 2, trbb[m, 1] + 2], str(m + 1), font = Font1, fill = (0,255,0))
                if type is 1:
                    draw.line([gtbb[m, 0], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1]], fill = (0,255,0), width = 6)
            else:
                draw.line([trbb[m, 0], trbb[m, 1], trbb[m, 0], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1] + trbb[m, 3], trbb[m, 0] + trbb[m, 2], trbb[m, 1], trbb[m, 0], trbb[m, 1]], fill = (255,0,0), width = 6)
                draw.text([trbb[m, 0] + 2, trbb[m, 1] + 2], str(m + 1), font = Font1, fill = (255,0,0))
                if type is 1:
                    draw.line([gtbb[m, 0], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1] + gtbb[m, 3], gtbb[m, 0] + gtbb[m, 2], gtbb[m, 1], gtbb[m, 0], gtbb[m, 1]], fill = (255,0,0), width = 6)
                
    del draw
    image.save(os.path.join(savefig_dir,'%04d.jpg'%(ind)), "JPEG")
    del image

In [13]:
def filter_overlap(boxes):
    init_bbox_sc = boxes[:,4]
    boxes = boxes[np.argsort(-init_bbox_sc), :]
    for i in range(boxes.shape[0] - 1):
        if boxes[i,0] != -1:
            for j in range(i + 1, boxes.shape[0]):
                if(overlap_ratio(boxes[i,2:6], boxes[j,2:6]) >= 0.25):
                    boxes[j,0] = -1
    boxes = boxes[boxes[:,0] != -1, :]
    return boxes

In [14]:
def multidim_intersect(arr1, arr2):
#     print (arr1.dtype, arr2.dtype)
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected = np.intersect1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])

In [15]:
def multidim_setdiff(arr1, arr2):
#     print (arr1.dtype, arr2.dtype)
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected = np.setdiff1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])

Create a coordinate recorder for every car

In [16]:
def run_mdnet(img_list, init_bbox, gt=None, savefig_dir='', display=False):
    start = 20
    end = 1000
    end = min(len(img_list), end)
    init_bbox = filter_overlap(gt[gt[:,1] == start])
    init_bbox = init_bbox[:,2:6]
    M = init_bbox.shape[0]
    coordinate_cluster_all_object=[]
    coordinate_cluster_single_object=[]
    
 
    # Init bbox
    target_bbox = np.array(init_bbox)
    result = np.zeros((M,len(img_list),4))
    result_bb = np.zeros((M,len(img_list),4))
    result[:,start,:] = target_bbox
    result_bb[:,start,:] = target_bbox
    justifi = np.ones((M))
    justifi_no = np.zeros((M))
    justifiF = np.ones((M))
    track_type = np.ones((M))

    # Init model
    Model = []
    Success_fra = []
    Pos_feats_all = []
    Neg_feats_all = []
    Sample_generator = []
    Pos_generator = []
    Neg_generator = []
    Update_optimizer = []
    Target_score = []
    
    tic = time.time()
    criterion = BinaryLoss()
    # Load first image
    image = Image.open(img_list[start]).convert('RGB')
    
    for m in range(init_bbox.shape[0]):
        model = MDNet(opts['model_path'])
        if opts['use_gpu']:
            model = model.cuda()
        model.set_learnable_params(opts['ft_layers'])
    
        # Init criterion and optimizer 
#         criterion = BinaryLoss()
        init_optimizer = set_optimizer(model, opts['lr_init'])
        update_optimizer = set_optimizer(model, opts['lr_update'])
    
    
        
    
        # Train bbox regressor
        bbreg_examples = gen_samples(SampleGenerator('uniform', image.size, 0.3, 1.5, 1.1),
                                     target_bbox[m,:], opts['n_bbreg'], opts['overlap_bbreg'], opts['scale_bbreg'])
        bbreg_feats = forward_samples(model, image, bbreg_examples)
        bbreg = BBRegressor(image.size)
        bbreg.train(bbreg_feats, bbreg_examples, target_bbox[m,:])
    
        # Draw pos/neg samples
    
        pos_examples = gen_samples(SampleGenerator('gaussian', image.size, 0.1, 1.2),
                                   target_bbox[m,:], opts['n_pos_init'], opts['overlap_pos_init'])

        neg_examples = np.concatenate([
                        gen_samples(SampleGenerator('uniform', image.size, 1, 2, 1.1), 
                                    target_bbox[m,:], opts['n_neg_init']//2, opts['overlap_neg_init']),
                        gen_samples(SampleGenerator('whole', image.size, 0, 1.2, 1.1),
                                    target_bbox[m,:], opts['n_neg_init']//2, opts['overlap_neg_init'])])
        neg_examples = np.random.permutation(neg_examples)

        # Extract pos/neg features
        pos_feats = forward_samples(model, image, pos_examples)
        neg_feats = forward_samples(model, image, neg_examples)
        feat_dim = pos_feats.size(-1)

        # Initial training
        train(model, criterion, init_optimizer, pos_feats, neg_feats, opts['maxiter_init'])
    
        # Init sample generators
        sample_generator = SampleGenerator('gaussian', image.size, opts['trans_f'], opts['scale_f'], valid=True)
        pos_generator = SampleGenerator('gaussian', image.size, 0.1, 1.2)
        neg_generator = SampleGenerator('uniform', image.size, 1.5, 1.2)

        # Init pos/neg features for update
        pos_feats_all = [pos_feats[:opts['n_pos_update']]]
        neg_feats_all = [neg_feats[:opts['n_neg_update']]]
        Model.append(model)
        success_fra = [start]
        Success_fra.append(success_fra)
        Pos_feats_all.append(pos_feats_all)
        Neg_feats_all.append(neg_feats_all)
        Sample_generator.append(sample_generator)
        Pos_generator.append(pos_generator)
        Neg_generator.append(neg_generator)
        Update_optimizer.append(update_optimizer)
        Target_score.append(2.)
        
        # record x0,y0 for each object in first frame
        temp = np.array((init_bbox[m,0]+init_bbox[m,2]/2,init_bbox[m,1]+init_bbox[m,3]/2))
        coordinate_cluster_single_object.append(temp)
        coordinate_cluster_all_object.append(coordinate_cluster_single_object)
        coordinate_cluster_single_object=[]
               
        #print (type(temp))
        #print (temp.shape)
        #print (temp)
        
        
        del model, success_fra, pos_feats_all, neg_feats_all, sample_generator, pos_generator, neg_generator, update_optimizer
        
         
    spf_total = time.time()-tic


    # Display
    savefig = savefig_dir != ''
    if display or savefig: 
#         dpi = 80.0
#         figsize = (image.size[0]/dpi, image.size[1]/dpi)

#         fig = plt.figure(frameon=False, figsize=figsize, dpi=dpi)
#         ax = plt.Axes(fig, [0., 0., 1., 1.])
#         ax.set_axis_off()
#         fig.add_axes(ax)
#         im = ax.imshow(image, aspect='equal')#, aspect='normal'

#         if gt is not None:
#             gt_rect = plt.Rectangle(tuple(gt[0,:2]),(gt[0,2]),(gt[0,3]), 
#                     linewidth=3, edgecolor="#00ff00", zorder=1, fill=False)
#             ax.add_patch(gt_rect)
       
#         rect = plt.Rectangle(tuple((result_bb[0,:2])),(result_bb[0,2]),(result_bb[0,3]), 
#                 linewidth=3, edgecolor="#ff0000", zorder=1, fill=False)
#         ax.add_patch(rect)

#         if display:
#             plt.pause(.01)
#             plt.draw()
        if savefig:
            save_fig(image, result_bb[:,start,:], gt[gt[:,1] == start], start, justifi, justifiF, type = 0)
            #fig.savefig(os.path.join(savefig_dir,'0000.jpg'),dpi=dpi)#
    print str(start)
    
    # Main loop
    for i in range(start + 1,end):#len(img_list)
        tic = time.time()
        # Load image
        image = Image.open(img_list[i]).convert('RGB')
        cur_det = filter_overlap(gt[gt[:,1] == i,:])
        cur_det = cur_det[:,2:6]
        for m in range(M):
            if justifiF[m] == 0:
                continue
            success = False
            # Estimate target bbox
            ratio_dec = overlap_ratio(cur_det, target_bbox[m,:])
            cur_can = cur_det[ratio_dec > 0.3, :]
            if(cur_can.shape[0] > 0):
                sample_scores = forward_samples(Model[m], image, cur_can, out_layer='fc6')
                top_scores, top_idx = sample_scores[:,1].topk(1)
                top_idx = top_idx[0]#.cpu().numpy()
                Target_score[m] = top_scores[0]#.cpu().numpy()
            else:
                Target_score[m] = -1
                
            if(Target_score[m] > 0.5):
                success = True
                track_type[m] = 1
                target_bbox[m,:] = cur_can[top_idx]
                result[m,i,:] = target_bbox[m,:]
                result_bb[m,i,:] = target_bbox[m,:]
                
                                   
            else:
                track_type[m] = 2
                samples = gen_samples(Sample_generator[m], target_bbox[m,:], opts['n_samples'])
                sample_scores = forward_samples(Model[m], image, samples, out_layer='fc6')
                top_scores, top_idx = sample_scores[:,1].topk(2)
                top_idx = top_idx.cpu().numpy()
                Target_score[m] = top_scores.mean()
                target_bbox[m,:] = samples[top_idx].mean(axis=0)
            
                success = Target_score[m] > 1. # opts['success_thr']
                median = Target_score[m] > 0.
            
                # Expand search area at failure
                if success:
                    Sample_generator[m].set_trans_f(opts['trans_f'])
                elif median:
                    Sample_generator[m].set_trans_f(min(1.05 * Sample_generator[m].get_trans_f(), 0.5 * (opts['trans_f'] + opts['trans_f_expand'])))
                else:
                    Sample_generator[m].set_trans_f(min(1.1 * Sample_generator[m].get_trans_f(), opts['trans_f_expand']))
                    

                # Bbox regression
                if success:
                    bbreg_samples = samples[top_idx]
                    bbreg_feats = forward_samples(Model[m], image, bbreg_samples)
                    bbreg_samples = bbreg.predict(bbreg_feats, bbreg_samples)
                    bbreg_bbox = bbreg_samples.mean(axis=0)
                    del bbreg_samples
                else:
                    bbreg_bbox = target_bbox[m,:]
        
                # Copy previous result at failure
#                 if not success:
#                     target_bbox[m,:] = result[m,i - 1,:]
#                     bbreg_bbox = result_bb[m,i - 1,:]

                # Save result
                result[m,i,:] = target_bbox[m,:]
                result_bb[m,i,:] = bbreg_bbox
                
                
            
            # Double check module
            if int(track_type[m]) != 1 and justifi[m] == 0 and i > 20:
                undet = 0
                for dc in range(min(20, int(i - Success_fra[m][0]))):
                    dc_cur_det = filter_overlap(gt[gt[:,1] == i - dc,:])[:,2:6]
                    ratio_dc = overlap_ratio(dc_cur_det, result[m,i - dc,:])
                    ratio_dc = ratio_dc > 0.05
                    if np.sum(ratio_dc) < 1:
                        undet = undet + 1
                    else:
                        break
                if undet > 5:
                    justifiF[m] = 0
                    for pre in range(int(justifi_no[m])):
                        result[m,i - pre,:] = np.zeros((4))
                    Model[m] = []
                    Pos_feats_all[m] = []
                    Neg_feats_all[m] = []
                    Sample_generator[m] = []
                    Pos_generator[m] = []
                    Neg_generator[m] = []
                    Update_optimizer[m] = []
                    continue
            
            if success:
                Success_fra[m].append(i)
                justifi[m] = 1
                justifi_no[m] = 0
                coordinate_cluster_all_object[m].append(np.array((result_bb[m,i,0]+result_bb[m,i,2]/2,result_bb[m,i,1]+result_bb[m,i,3]/2)))
            elif median:
                justifi[m] = 1
                justifi_no[m] = 0
            else:
                xmin = result[m,i - 1,0]
                ymin = result[m,i - 1,1]
                xmax = result[m,i - 1,0] + result[m,i - 1,2]
                ymax = result[m,i - 1,1] + result[m,i - 1,3]
                width, height = image.size
                if (xmin <= 1 or ymin <= 1 or xmax >= width - 1 or ymax >= height - 1) and Target_score[m] <= 0:
                    justifiF[m] = 0
                    justifi[m] = 0
                    Model[m] = []
                    Pos_feats_all[m] = []
                    Neg_feats_all[m] = []
                    Sample_generator[m] = []
                    Pos_generator[m] = []
                    Neg_generator[m] = []
                    Update_optimizer[m] = []
                    continue
                else:
                    if justifi_no[m] < 20 and i != end - 1:
                        justifi_no[m] = justifi_no[m] + 1
                        justifi[m] = 0
                    else:
                        justifiF[m] = 0
                        for pre in range(int(justifi_no[m])):
                            result[m,i - pre,:] = np.zeros((4))
                        Model[m] = []
                        Pos_feats_all[m] = []
                        Neg_feats_all[m] = []
                        Sample_generator[m] = []
                        Pos_generator[m] = []
                        Neg_generator[m] = []
                        Update_optimizer[m] = []
                        continue

            # Data collect
            if success:
                # Draw pos/neg samples
                pos_examples = gen_samples(Pos_generator[m], target_bbox[m,:], 
                                           opts['n_pos_update'],
                                           opts['overlap_pos_update'])
                neg_examples = gen_samples(Neg_generator[m], target_bbox[m,:], 
                                           opts['n_neg_update'],
                                           opts['overlap_neg_update'])

                # Extract pos/neg features
                pos_feats = forward_samples(Model[m], image, pos_examples)
                neg_feats = forward_samples(Model[m], image, neg_examples)
                Pos_feats_all[m].append(pos_feats)
                Neg_feats_all[m].append(neg_feats)
                if len(Pos_feats_all[m]) > opts['n_frames_long']:
                    del Pos_feats_all[m][0]
                if len(Neg_feats_all[m]) > opts['n_frames_short']:
                    del Neg_feats_all[m][0]
                del pos_feats, neg_feats, pos_examples, neg_examples

            # Short term update
            if not success:
                nframes = min(opts['n_frames_short'],len(Neg_feats_all[m]))
                pos_data = torch.stack(Pos_feats_all[m][-nframes:],0).view(-1,feat_dim)
                neg_data = torch.stack(Neg_feats_all[m],0).view(-1,feat_dim)
                train(Model[m], criterion, Update_optimizer[m], pos_data, neg_data, opts['maxiter_update'])
                del pos_data, neg_data
        
            # Long term update
            elif i % opts['long_interval'] == 0:
                pos_data = torch.stack(Pos_feats_all[m],0).view(-1,feat_dim)
                neg_data = torch.stack(Neg_feats_all[m],0).view(-1,feat_dim)
                train(Model[m], criterion, Update_optimizer[m], pos_data, neg_data, opts['maxiter_update'])
                del pos_data, neg_data
        
        spf = time.time()-tic
        spf_total += spf
        
        print "Frame %d/%d, Time %.3f" % (i, len(img_list), spf)

        # Display
        if display or savefig:
#             im.set_data(image)

#             if gt is not None:
#                 gt_rect.set_xy(gt[i,:2])
#                 gt_rect.set_width(gt[i,2])
#                 gt_rect.set_height(gt[i,3])

#             rect.set_xy(result_bb[i,:2])
#             rect.set_width(result_bb[i,2])
#             rect.set_height(result_bb[i,3])
            
#             if display:
#                 plt.pause(.01)
#                 plt.draw()
            if savefig:
                save_fig_online(image, result_bb[:,i,:], gt[gt[:,2] == i], i, justifi, justifiF, type = 0, 
                                cluster = coordinate_cluster_all_object, thres = 20)
#                 #fig.savefig(os.path.join(savefig_dir,'%04d.jpg'%(i)))#,dpi=dpi

#         if gt is None:
#             print "Frame %d/%d, Score %.3f, Time %.3f" % \
#                 (i, len(img_list), target_score, spf)
#         else:
#             print "Frame %d/%d, Overlap %.3f, Score %.3f, Time %.3f" % \
#                 (i, len(img_list), overlap_ratio(gt[i],result_bb[i])[0], target_score, spf)
        if (i % 3 == 0) and (i >= 3):
            bbox_cur = result[:,i,:]
#             cur_com = multidim_intersect(cur_det.copy(), bbox_cur.copy())
#             print (cur_com, bbox_cur)
            cur_com = multidim_setdiff(cur_det.copy(), bbox_cur.copy())
            if cur_com.shape[0] > 0:
                print 'add'
                for bbox in cur_com:
                    ratio = overlap_ratio(bbox_cur, bbox)
                    ratio = -np.sort(-ratio)
                    if ratio[0] < 0.10:
                        M = M + 1
                        result = np.concatenate((result, np.zeros((1,len(img_list),4))), axis = 0)
                        result[M - 1,i,:] = bbox
                        result_bb = np.concatenate((result_bb, np.zeros((1,len(img_list),4))), axis = 0)
                        result_bb[M - 1,i,:] = bbox
                        target_bbox = np.concatenate((target_bbox, np.zeros((1,4))), axis = 0)
                        target_bbox[M - 1,:] = bbox
                        
                        temp = np.array((bbox[0]+bbox[2]/2,bbox[1]+bbox[3]/2))
                        coordinate_cluster_single_object.append(temp)
                        coordinate_cluster_all_object.append(coordinate_cluster_single_object)
                        coordinate_cluster_single_object=[]
                                              
                        justifi = np.concatenate((justifi, np.ones((1))), axis = 0)
                        track_type = np.concatenate((track_type, np.ones((1))), axis = 0)
                        justifiF = np.concatenate((justifiF, np.ones((1))), axis = 0)
                        justifi_no = np.concatenate((justifi_no, np.zeros((1))), axis = 0)
                        model = MDNet(opts['model_path'])
                        if opts['use_gpu']:
                            model = model.cuda()
                        model.set_learnable_params(opts['ft_layers'])
    
                        # Init criterion and optimizer 
                        init_optimizer = set_optimizer(model, opts['lr_init'])
                        update_optimizer = set_optimizer(model, opts['lr_update'])
    
    
                        # Load first image
                        image = Image.open(img_list[i]).convert('RGB')
    
                        # Train bbox regressor
                        bbreg_examples = gen_samples(SampleGenerator('uniform', image.size, 0.3, 1.5, 1.1),
                                                     target_bbox[M - 1,:], opts['n_bbreg'], opts['overlap_bbreg'], opts['scale_bbreg'])
                        bbreg_feats = forward_samples(model, image, bbreg_examples)
                        bbreg = BBRegressor(image.size)
                        bbreg.train(bbreg_feats, bbreg_examples, target_bbox[M - 1,:])
    
                        # Draw pos/neg samples
    
                        pos_examples = gen_samples(SampleGenerator('gaussian', image.size, 0.1, 1.2),
                                                   target_bbox[M - 1,:], opts['n_pos_init'], opts['overlap_pos_init'])

                        neg_examples = np.concatenate([
                                        gen_samples(SampleGenerator('uniform', image.size, 1, 2, 1.1), 
                                                    target_bbox[M - 1,:], opts['n_neg_init']//2, opts['overlap_neg_init']),
                                        gen_samples(SampleGenerator('whole', image.size, 0, 1.2, 1.1),
                                                    target_bbox[M - 1,:], opts['n_neg_init']//2, opts['overlap_neg_init'])])
                        neg_examples = np.random.permutation(neg_examples)

                        # Extract pos/neg features
                        pos_feats = forward_samples(model, image, pos_examples)
                        neg_feats = forward_samples(model, image, neg_examples)
                        feat_dim = pos_feats.size(-1)

                        # Initial training
                        train(model, criterion, init_optimizer, pos_feats, neg_feats, opts['maxiter_init'])
    
                        # Init sample generators
                        sample_generator = SampleGenerator('gaussian', image.size, opts['trans_f'], opts['scale_f'], valid=True)
                        pos_generator = SampleGenerator('gaussian', image.size, 0.1, 1.2)
                        neg_generator = SampleGenerator('uniform', image.size, 1.5, 1.2)

                        # Init pos/neg features for update
                        pos_feats_all = [pos_feats[:opts['n_pos_update']]]
                        neg_feats_all = [neg_feats[:opts['n_neg_update']]]
                        Model.append(model)
                        success_fra = [i]
                        Success_fra.append(success_fra)
                        Pos_feats_all.append(pos_feats_all)
                        Neg_feats_all.append(neg_feats_all)
                        Sample_generator.append(sample_generator)
                        Pos_generator.append(pos_generator)
                        Neg_generator.append(neg_generator)
                        Update_optimizer.append(update_optimizer)
                        Target_score.append(2.)
                        del model, success_fra, pos_feats_all, neg_feats_all, sample_generator, pos_generator, neg_generator, update_optimizer
                        
            del cur_com, bbox_cur
                        

    fps = len(img_list) / spf_total
    for i in range(len(Success_fra)):
        print (i, Success_fra[i])
        
        
    print(len(coordinate_cluster_all_object))
    print(len(coordinate_cluster_all_object[0]))
    for i in range(len(coordinate_cluster_all_object[0])):
        print(coordinate_cluster_all_object[i])
    return result, result_bb, fps

In [17]:
def gen_config_ju(seq, savefig = False, json = '', display = False):
    if seq != '':
        # generate config from a sequence name
        
        seq_home = '../dataset/OTB'
        save_home = '../result_fig'
        result_home = '../result'
        
        seq_name = seq
        img_dir = os.path.join(seq_home, seq_name, 'img')
        gt_path = os.path.join(seq_home, seq_name, 'dec.txt')

        img_list = os.listdir(img_dir)
        img_list.sort()
        img_list = [os.path.join(img_dir,x) for x in img_list]

        gt = np.loadtxt(gt_path,delimiter=',')
        init_bbox = gt[gt[:,1] == 0, :]
        
        savefig_dir = os.path.join(save_home,seq_name)
        result_dir = os.path.join(result_home,seq_name)
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
        result_path = os.path.join(result_dir,'result.json')

    elif json != '':
        # load config from a json file

        param = json.load(open(json,'r'))
        seq_name = param['seq_name']
        img_list = param['img_list']
        init_bbox = param['init_bbox']
        savefig_dir = param['savefig_dir']
        result_path = param['result_path']
        gt = None
        
    if savefig:
        if not os.path.exists(savefig_dir):
            os.makedirs(savefig_dir)
    else:
        savefig_dir = ''

    return img_list, init_bbox, gt, savefig_dir, display, result_path

In [18]:
# Generate sequence config
img_list, init_bbox, gt, savefig_dir, display, result_path = gen_config_ju('202', savefig = True)

In [19]:
# Run tracker
result, result_bb, fps = run_mdnet(img_list, init_bbox, gt=gt, savefig_dir=savefig_dir, display=display)
    
# Save result
res = {}
res['res'] = result_bb.round().tolist()
res['type'] = 'rect'
res['fps'] = fps
json.dump(res, open(result_path, 'w'), indent=2)

20
Frame 21/309, Time 0.730
Frame 22/309, Time 0.744
Frame 23/309, Time 0.757
Frame 24/309, Time 0.772
Frame 25/309, Time 0.779
Frame 26/309, Time 0.812
Frame 27/309, Time 0.838
Frame 28/309, Time 0.879
Frame 29/309, Time 0.927
sum_dis = 0.695650735595
sum_dis = 0.858287460692
sum_dis = 1.01669553843
Frame 30/309, Time 1.285
sum_dis = 0.910003070433
sum_dis = 0.795196240909
sum_dis = 0.924792267985
Frame 31/309, Time 1.000
sum_dis = 0.708564967244
sum_dis = 0.974809362392
sum_dis = 0.910744191661
Frame 32/309, Time 1.056
sum_dis = 0.739152712911
sum_dis = 1.14845499069
sum_dis = 0.898224791845
Frame 33/309, Time 1.145
sum_dis = 0.788644380909
sum_dis = 1.29238770334
sum_dis = 0.915713967493
Frame 34/309, Time 1.176
sum_dis = 1.41786664353
sum_dis = 0.939382625814
sum_dis = 0.881380634393
Frame 35/309, Time 1.196
sum_dis = 1.58167443718
sum_dis = 0.962805066967
sum_dis = 1.3745377684
Frame 36/309, Time 1.300
sum_dis = 1.05591360964
sum_dis = 1.11865126393
sum_dis = 1.0879589294
Frame 37

SyntaxError: broken PNG file (chunk '\x00\x00\x00\x00')